# 02: Cross-validation

## p28 binの確認

In [1]:
# stratified-kfold for regression 
import numpy as np 
import pandas as pd 
 
from sklearn import datasets 
from sklearn import model_selection 

In [2]:
# we create a sample dataset with 15000 samples  
# and 100 features and 1 target 
X, y = datasets.make_regression( 
    n_samples=15000, n_features=100, n_targets=1 
) 
 
# create a dataframe out of our numpy arrays 
df = pd.DataFrame( 
    X, 
    columns=[f"f_{i}" for i in range(X.shape[1])] 
) 
df.loc[:, "target"] = y 

In [3]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target
0,-0.725536,-0.510135,-1.197524,-0.709489,0.171271,0.014377,1.594370,-0.156228,-1.134739,0.631922,...,-0.826296,-0.266531,1.272600,0.623143,-0.786605,0.856343,0.163177,0.081595,0.771702,31.243463
1,0.515579,1.866164,1.026614,0.252979,-0.372898,0.600146,-0.387818,-0.500411,-0.124250,-0.417828,...,-2.759727,-0.424287,-1.607841,-1.396671,-0.563047,3.528111,0.161747,-1.057309,1.229461,116.111726
2,0.678059,0.727006,-0.124652,-0.266601,2.155618,0.287473,-1.924864,-0.534405,-0.315248,-0.018049,...,-0.397058,-0.279611,-0.122944,-0.330166,-0.250941,-1.039255,0.912719,-0.014749,-1.025483,91.746839
3,0.346841,1.382796,0.168493,-0.809429,0.783522,0.108018,0.545448,-0.133110,-0.769088,0.111707,...,-1.197373,1.246227,-0.721057,-1.002684,-0.887223,-0.479859,0.399176,0.486740,-3.483927,184.561445
4,0.509762,1.559858,-0.544077,-1.114001,-2.186983,-0.874338,-0.747612,0.657812,0.021387,-0.703442,...,0.981046,0.895436,0.609510,-1.491491,-1.428940,-2.692509,-1.436567,0.485475,-0.953060,104.704473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.129682,0.838498,0.132090,1.351785,0.794400,1.313001,2.031403,-0.158750,0.162589,-0.259710,...,1.798974,-0.022976,-0.822264,1.845741,0.332887,0.932269,-1.636342,1.104695,-0.022178,-198.842301
14996,1.364773,1.653099,1.668593,-1.118883,1.663941,-0.840597,-0.283655,0.795721,-0.889434,1.454678,...,0.535393,0.219381,0.549666,-0.134882,-2.059756,0.275079,-1.123286,-0.370264,-0.345349,-533.474644
14997,0.382241,-1.164177,0.941359,-1.781853,0.062186,-0.951935,0.582933,-0.748317,0.104242,0.166324,...,-0.830808,-0.990055,-0.628456,0.707491,0.586865,-0.473959,0.997593,1.869571,-0.070876,-167.062888
14998,0.339518,-1.100069,0.492697,-0.829690,-0.893600,0.901667,-0.646298,-0.397993,0.147721,-0.240519,...,0.758861,-0.619864,0.029768,-0.470911,0.722793,0.045131,0.596014,0.536028,0.832623,189.764478


In [4]:
# we create a new column called kfold and fill it with -1 
df["kfold"] = -1 

In [5]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold
0,-0.725536,-0.510135,-1.197524,-0.709489,0.171271,0.014377,1.594370,-0.156228,-1.134739,0.631922,...,-0.266531,1.272600,0.623143,-0.786605,0.856343,0.163177,0.081595,0.771702,31.243463,-1
1,0.515579,1.866164,1.026614,0.252979,-0.372898,0.600146,-0.387818,-0.500411,-0.124250,-0.417828,...,-0.424287,-1.607841,-1.396671,-0.563047,3.528111,0.161747,-1.057309,1.229461,116.111726,-1
2,0.678059,0.727006,-0.124652,-0.266601,2.155618,0.287473,-1.924864,-0.534405,-0.315248,-0.018049,...,-0.279611,-0.122944,-0.330166,-0.250941,-1.039255,0.912719,-0.014749,-1.025483,91.746839,-1
3,0.346841,1.382796,0.168493,-0.809429,0.783522,0.108018,0.545448,-0.133110,-0.769088,0.111707,...,1.246227,-0.721057,-1.002684,-0.887223,-0.479859,0.399176,0.486740,-3.483927,184.561445,-1
4,0.509762,1.559858,-0.544077,-1.114001,-2.186983,-0.874338,-0.747612,0.657812,0.021387,-0.703442,...,0.895436,0.609510,-1.491491,-1.428940,-2.692509,-1.436567,0.485475,-0.953060,104.704473,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.129682,0.838498,0.132090,1.351785,0.794400,1.313001,2.031403,-0.158750,0.162589,-0.259710,...,-0.022976,-0.822264,1.845741,0.332887,0.932269,-1.636342,1.104695,-0.022178,-198.842301,-1
14996,1.364773,1.653099,1.668593,-1.118883,1.663941,-0.840597,-0.283655,0.795721,-0.889434,1.454678,...,0.219381,0.549666,-0.134882,-2.059756,0.275079,-1.123286,-0.370264,-0.345349,-533.474644,-1
14997,0.382241,-1.164177,0.941359,-1.781853,0.062186,-0.951935,0.582933,-0.748317,0.104242,0.166324,...,-0.990055,-0.628456,0.707491,0.586865,-0.473959,0.997593,1.869571,-0.070876,-167.062888,-1
14998,0.339518,-1.100069,0.492697,-0.829690,-0.893600,0.901667,-0.646298,-0.397993,0.147721,-0.240519,...,-0.619864,0.029768,-0.470911,0.722793,0.045131,0.596014,0.536028,0.832623,189.764478,-1


In [6]:
# the next step is to randomize the rows of the data 
df = df.sample(frac=1).reset_index(drop=True) 

In [7]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold
0,0.861786,0.477147,-2.873871,0.861089,0.277401,1.108156,-0.955897,1.241708,-1.261578,-0.148962,...,-0.086912,0.362248,-1.104756,-0.817061,0.156769,0.474505,0.622152,-1.073786,-359.122221,-1
1,1.689787,1.608610,-0.247553,0.877061,0.414011,-0.326255,-1.483898,-1.560423,0.952403,-1.358833,...,0.972545,0.685132,1.660361,-1.679775,-1.051254,0.036897,-0.730462,-0.587713,-406.598345,-1
2,-0.180440,-0.315380,0.040100,0.668596,-0.055357,-2.394821,1.067345,0.590214,0.306079,-1.597808,...,1.753452,0.516651,0.364588,-0.817797,-0.533618,1.664789,-1.166612,0.147601,-118.598114,-1
3,0.293877,-0.868486,2.155222,0.072913,0.170806,0.394134,0.379993,-0.224103,2.273163,0.370696,...,-2.186759,-0.837805,-1.895062,0.494189,-1.279694,-0.650972,-1.103563,1.034213,28.997119,-1
4,0.242058,0.870708,-0.516519,-2.460833,0.062883,0.173496,-0.879582,0.232831,-0.779083,-0.430250,...,-0.433548,-1.483720,-0.081120,-1.285276,0.186004,-0.104175,-0.142255,0.749897,33.003522,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.311344,0.882880,1.187982,-1.220254,0.097312,-0.602364,0.926719,-2.215464,0.030975,0.301784,...,0.909168,1.081391,-1.430585,0.483392,0.818886,-0.072983,0.893851,0.873796,114.736237,-1
14996,0.214487,1.408579,0.532776,-0.512852,-0.014906,0.134701,0.026999,-0.734997,-0.387580,-0.970540,...,-0.997671,0.850019,0.210704,0.466033,0.393370,-0.225699,-0.244383,2.871019,-185.762451,-1
14997,0.651077,-0.519529,-1.793541,0.913655,-0.415214,0.407683,1.842905,-0.332982,0.286900,0.546470,...,-2.012399,-0.148244,-0.507126,2.294706,2.050424,-1.277500,-1.540499,0.283773,-300.567756,-1
14998,-1.452831,-0.801327,0.554605,1.246748,0.293761,0.820510,0.801424,0.125446,0.801768,-1.458160,...,0.315865,-0.694049,-0.448445,-1.123416,-0.157052,0.743551,0.310716,1.638402,-12.431634,-1


In [8]:
# calculate the number of bins by Sturge's rule 
# I take the floor of the value, you can also 
# just round it 
num_bins = np.floor(1 + np.log2(len(df))) 

In [9]:
num_bins

14.0

In [10]:
# bin targets
# https://note.nkmk.me/python-pandas-cut-qcut-binning/
df.loc[:, "bins"] = pd.cut(df["target"], bins=int(num_bins), labels=False) # int(num_bins) にして整数型に変更

In [11]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold,bins
0,0.861786,0.477147,-2.873871,0.861089,0.277401,1.108156,-0.955897,1.241708,-1.261578,-0.148962,...,0.362248,-1.104756,-0.817061,0.156769,0.474505,0.622152,-1.073786,-359.122221,-1,3
1,1.689787,1.608610,-0.247553,0.877061,0.414011,-0.326255,-1.483898,-1.560423,0.952403,-1.358833,...,0.685132,1.660361,-1.679775,-1.051254,0.036897,-0.730462,-0.587713,-406.598345,-1,3
2,-0.180440,-0.315380,0.040100,0.668596,-0.055357,-2.394821,1.067345,0.590214,0.306079,-1.597808,...,0.516651,0.364588,-0.817797,-0.533618,1.664789,-1.166612,0.147601,-118.598114,-1,5
3,0.293877,-0.868486,2.155222,0.072913,0.170806,0.394134,0.379993,-0.224103,2.273163,0.370696,...,-0.837805,-1.895062,0.494189,-1.279694,-0.650972,-1.103563,1.034213,28.997119,-1,7
4,0.242058,0.870708,-0.516519,-2.460833,0.062883,0.173496,-0.879582,0.232831,-0.779083,-0.430250,...,-1.483720,-0.081120,-1.285276,0.186004,-0.104175,-0.142255,0.749897,33.003522,-1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.311344,0.882880,1.187982,-1.220254,0.097312,-0.602364,0.926719,-2.215464,0.030975,0.301784,...,1.081391,-1.430585,0.483392,0.818886,-0.072983,0.893851,0.873796,114.736237,-1,7
14996,0.214487,1.408579,0.532776,-0.512852,-0.014906,0.134701,0.026999,-0.734997,-0.387580,-0.970540,...,0.850019,0.210704,0.466033,0.393370,-0.225699,-0.244383,2.871019,-185.762451,-1,5
14997,0.651077,-0.519529,-1.793541,0.913655,-0.415214,0.407683,1.842905,-0.332982,0.286900,0.546470,...,-0.148244,-0.507126,2.294706,2.050424,-1.277500,-1.540499,0.283773,-300.567756,-1,4
14998,-1.452831,-0.801327,0.554605,1.246748,0.293761,0.820510,0.801424,0.125446,0.801768,-1.458160,...,-0.694049,-0.448445,-1.123416,-0.157052,0.743551,0.310716,1.638402,-12.431634,-1,6


In [12]:
# initiate the kfold class from model_selection module 
kf = model_selection.StratifiedKFold(n_splits=5) 

In [13]:
# fill the new kfold column 
# note that, instead of targets, we use bins! 
for f, (t_, v_) in enumerate(kf.split(X=df, y=df.bins.values)): 
    df.loc[v_, 'kfold'] = f 

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [14]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold,bins
0,0.861786,0.477147,-2.873871,0.861089,0.277401,1.108156,-0.955897,1.241708,-1.261578,-0.148962,...,0.362248,-1.104756,-0.817061,0.156769,0.474505,0.622152,-1.073786,-359.122221,0,3
1,1.689787,1.608610,-0.247553,0.877061,0.414011,-0.326255,-1.483898,-1.560423,0.952403,-1.358833,...,0.685132,1.660361,-1.679775,-1.051254,0.036897,-0.730462,-0.587713,-406.598345,0,3
2,-0.180440,-0.315380,0.040100,0.668596,-0.055357,-2.394821,1.067345,0.590214,0.306079,-1.597808,...,0.516651,0.364588,-0.817797,-0.533618,1.664789,-1.166612,0.147601,-118.598114,0,5
3,0.293877,-0.868486,2.155222,0.072913,0.170806,0.394134,0.379993,-0.224103,2.273163,0.370696,...,-0.837805,-1.895062,0.494189,-1.279694,-0.650972,-1.103563,1.034213,28.997119,0,7
4,0.242058,0.870708,-0.516519,-2.460833,0.062883,0.173496,-0.879582,0.232831,-0.779083,-0.430250,...,-1.483720,-0.081120,-1.285276,0.186004,-0.104175,-0.142255,0.749897,33.003522,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.311344,0.882880,1.187982,-1.220254,0.097312,-0.602364,0.926719,-2.215464,0.030975,0.301784,...,1.081391,-1.430585,0.483392,0.818886,-0.072983,0.893851,0.873796,114.736237,4,7
14996,0.214487,1.408579,0.532776,-0.512852,-0.014906,0.134701,0.026999,-0.734997,-0.387580,-0.970540,...,0.850019,0.210704,0.466033,0.393370,-0.225699,-0.244383,2.871019,-185.762451,4,5
14997,0.651077,-0.519529,-1.793541,0.913655,-0.415214,0.407683,1.842905,-0.332982,0.286900,0.546470,...,-0.148244,-0.507126,2.294706,2.050424,-1.277500,-1.540499,0.283773,-300.567756,4,4
14998,-1.452831,-0.801327,0.554605,1.246748,0.293761,0.820510,0.801424,0.125446,0.801768,-1.458160,...,-0.694049,-0.448445,-1.123416,-0.157052,0.743551,0.310716,1.638402,-12.431634,4,6


In [15]:
df.to_csv('./data/df1.csv')